<a href="https://colab.research.google.com/github/giahuynguyen28/lab1/blob/master/TestRank_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# Upload the CSV file
from google.colab import files
uploaded = files.upload()

Saving tennis_articles_v4.csv to tennis_articles_v4.csv


In [3]:
# Read the CSV file
import io
df = pd.read_csv(io.StringIO(uploaded['tennis_articles_v4.csv'].decode("utf-8")))

In [4]:
df.head()

,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [8]:
import nltk
nltk.download('punkt_tab')
sentences = []
for s in df['article_text']:
  sentences.append(sent_tokenize(s))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [9]:
# flatten the list
sentences = [y for x in sentences for y in x]

In [10]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [11]:
nltk.download('stopwords')# one time execution

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [14]:
# function to remove stopwords
def remove_stopwords(sen):
  sen_new = " ".join([i for i in sen if i not in stop_words])
  return sen_new

In [15]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [16]:
# download pretrained GloVe word embeddings
! wget http://nlp.stanford.edu/data/glove.6B.zip

--2024-11-19 01:06:53--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-11-19 01:06:53--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-11-19 01:06:53--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [17]:
! unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [18]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [19]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [20]:
len(sentence_vectors)

119

The next step is to find similarities among the sentences. We will use cosine similarity to find similarity between a pair of sentences. Let's create an empty similarity matrix for this task and populate it with cosine similarities of the sentences.

In [21]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [48]:
def mypagerank(sim_mat, d=0.85, max_iter=100000, tol=1e-6):
    n = sim_mat.shape[0]  # Số lượng câu (đỉnh)

    # Chuẩn hóa ma trận tương đồng (chuyển về xác suất)
    for i in range(n):
        if sim_mat[i].sum() != 0:
            sim_mat[i] /= sim_mat[i].sum()  # Chuẩn hóa hàng để thành xác suất

    # Khởi tạo PageRank cho các đỉnh
    scores = np.ones(n) / n  # Bắt đầu với giá trị đồng đều

    # Thuật toán lặp
    for _ in range(max_iter):
        prev_scores = scores.copy()
        for i in range(n):
            # Tính WS(V_i) theo công thức
            scores[i] = (1 - d) + d * np.sum(sim_mat[:, i] * prev_scores)

        # Kiểm tra điều kiện hội tụ
        if np.linalg.norm(scores - prev_scores, ord=1) < tol:
            break

    return scores;

In [49]:
scores = mypagerank(sim_mat)

In [51]:
# Pairing sentences with their scores
sentence_score_pairs = list(zip(clean_sentences, scores))

# Sorting the pairs based on the scores in descending order
sorted_sentence_score_pairs = sorted(sentence_score_pairs, key=lambda x: x[1], reverse=True)

# Printing the sorted sentences in order of importance
for i, (sentence, score) in enumerate(sorted_sentence_score_pairs[:20]):  # Printing top 10 sentences
    print(f"Rank {i+1}: {sentence} (Score: {score})\n")


Rank 1: major players feel big event late november combined one january australian open mean much tennis little rest. (Score: 1.1242528726094918)

Rank 2: speaking swiss indoors tournament play sundays final romanian qualifier marius copil, world number three said given impossibly short time frame make decision, opted commitment. (Score: 1.1195465046987114)

Rank 3: "i felt like best weeks get know players playing fed cup weeks olympic weeks, necessarily tournaments. (Score: 1.1169442583869658)

Rank 4: i'm courts i'm court playing, i'm competitor want beat every single person whether they're locker room across net.so i'm one strike conversation weather know next minutes go try win tennis match. (Score: 1.1164093485055466)

Rank 5: federer easier time previous match medvedev, three-setter shanghai two weeks ago. (Score: 1.1101290587045094)

Rank 6: spaniard broke anderson twice second get another chance south african's serve final set. (Score: 1.1056104565681495)

Rank 7: used first br

In [45]:
# Specify number of sentences to form the summary
sn = 10

# Generate summary
for i in range(sn):
  print(ranked_sentences[i][1] )

Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.
Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.
"I felt like the best weeks that I had to get to know players when I was playing were the Fed Cup weeks or the Olympic weeks, not necessarily during the tournaments.
When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.
Federer had an easier time than in his only previous match against Medvedev, a three-setter at Shanghai two weeks ago.
The S

Find the original article here https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/